- 추천시스템에서는 컨설팅 프로젝트나 문서 예시가 없다. 스파크와 추천 시스템은 사용하기 편리해서 시험에 적합한 대상이 아니기 때문이다. 추천 시스템을 파이썬과 스파크로 구축할 때 파이썬과 스파크 내에서 해결할 수 없는 문제가 있다. 유저를 모아야 하고, 피드백을 유용한 방법으로 수렴하며, 데이터를 정리해야 한다. 하지만 그런 기반을 만드는 건 스파크와 파이썬의 범위 밖의 일이다. <br>
스파크의 장점은 데이터 형식을 잘 받아들이고, 추천 시스템을 빨리 만드는 것이다. 데이터를 얻고 포맷하면서 수반되는 일들이 관건이다. 생각보다 간단해서 컨설팅 프로젝트나 문서 예시가 없다. <br>
- 더 깊은 공부는 아마존에서 구매할 수 있는 'Recommender Systems' 책으로 하는 것을 추천한다. 수학적 내용과 배경지식을 자세하게 다룬다.<br>
- 추천 시스템의 가장 일반적인 방법은 콘텐츠 기반과 협업 필터링(CF)이다.  협업 필터링은 항목에 대한 사용자의 태도 관련 지식을 활용해서 추천한다. 즉, 집단 지성을 이용해서 추천하는 것이다. 콘텐츠 기반 추천 시스템은 항목의 특성을 중심으로 추천한다. 항목간 유사성에 집중하는 것이다. 보통 협업 필터링이 자주 쓰인다. 결과가 더 좋고 대체로 구현하기 쉽기 때문이다. <br>
- spark.ml은 현재 모델 기반 협엽 필터링을 지원한다. <br>
- 파이썬과 스파크를 쓸 때 어떻게 되는지 보자. 세 명의 고객을 가정한다. 3개의 영화도 가정한다. 고객 1,2는 영화 A,B를 보고 5점 만점에 5점을 줬다. C는 1점만 줬다. 어떤 영화도 보지 않은 고객 3가 A영화를 보고 5점을 줬다. 협엽 필터링 모델은 고객3에게 영화 B를 추천할 것이다. 고객 3와 비슷하게 고객1,2도 영화 A,B를 좋아했기 때문이다. 직관적으로 이해하기 쉽다. 콘텐츠 기반 시스템은 사용자를 고려할 필요가 없다. 영화만 특성으로 분류하면 된다. 이런 특성에는 러닝타임, 장르와 출연한 배우 등이 있다. 이런 비슷한 특성을 가진 영화들끼리 분류해서 그룹 내 영화는 서로 비슷하게 만드는 것이다. 두 방법을 모두 활용한 추천 시스템도 자주 볼 수 있다.

# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [7]:
# Smaller dataset so we will use 0.8 / 0.2
# 데이터가 적어 0.8/0.2 를 사용하였다.
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
# 주요인자는 userCol, itemCol, ratingCol이다.
# 파이썬과 스파크를 활용해 추천 시스템을 만들 때 큰 비중을 차지하는 과정은
# 데이터를 사용자, 항목, 평가를 포함한 특정한 형식으로 만드는 것이다.
# maxIter, regParam에 대해서는 위에서 추천한 책으로 공부할 수 있다.
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", 
          ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()
# 해석해본다
# userId가 6인 사용자가 movieId가 1인 영화를 보고 1점을 매겼는데
# 모델은 1.067점을 매길것으로 예측한다.
# 괜찮은 예측이다.
# 물론 형편없는 예측도 있다.

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|     6| 1.0669236|
|      1|   1.0|    18|  2.412419|
|      6|   1.0|     1| 0.6057124|
|      6|   1.0|    13| 1.5337529|
|      6|   1.0|     2|0.34447113|
|      3|   1.0|     1| 1.9232892|
|      3|   1.0|     9| 2.0778308|
|      3|   1.0|    17| 2.3768747|
|      3|   2.0|     8| 2.9335582|
|      5|   1.0|     6| 1.8100878|
|      5|   1.0|     9|0.35960156|
|      5|   1.0|    14| 1.1813008|
|      4|   4.0|    26| 2.9325604|
|      4|   2.0|     1| 0.5961519|
|      4|   2.0|    13| 1.5668327|
|      4|   3.0|     2|-0.3433052|
|      7|   1.0|    10| 2.9161596|
|      7|   1.0|    18| 1.9083111|
|      2|   4.0|    28| 2.0951216|
|      2|   2.0|     1| 2.3257396|
+-------+------+------+----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
# 별점이 1~5점인것을 감안하면 엉망이다.
# 데이터셋이 작아서 당연한 결과이다.

Root-mean-square error = 1.902566316934313


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [12]:
# 새로운 사용자에게 적용하는 방법
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     12|    11|
|     13|    11|
|     21|    11|
|     32|    11|
|     38|    11|
|     47|    11|
|     51|    11|
|     59|    11|
|     70|    11|
|     75|    11|
|     76|    11|
|     79|    11|
|     88|    11|
|     89|    11|
|     94|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     13|    11| 4.2984686|
|     10|    11| 3.3747225|
|     32|    11|  3.326592|
|     76|    11| 3.2773159|
|     94|    11|  2.830971|
|     47|    11| 2.7540476|
|     75|    11|  2.744392|
|     38|    11| 2.6029787|
|     12|    11| 2.4413304|
|     21|    11| 2.3178768|
|     51|    11|  2.226689|
|     89|    11| 2.1649518|
|     59|    11| 1.8857139|
|     88|    11| 1.7154149|
|     79|    11|  1.060319|
|     70|    11| 0.6225053|
+-------+------+----------+



콜드 스타트 문제가 있을 수 있다. <br>
사용자가 플랫폼에 방금 가입해서 영화를 본 적 없을 때 발생한다. 이런 경우 어떤 영화를 봤는지 설문 조사를 해서 해결할 수 있다.

# Great Job!